In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('fake-news/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
###Drop Nan Values
df=df.dropna()

In [5]:
## Get the Independent Features
X = df.drop('label',axis=1)

In [6]:
## Get the Dependent features
y=df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
import tensorflow as tf

In [10]:
tf.__version__

'2.3.0'

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense

In [12]:

### Vocabulary size
voc_size=5000

## Onehot Representation 

In [13]:
messages = X.copy()

In [14]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [15]:
messages.reset_index(inplace=True)

In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [19]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [20]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[657, 4677, 249, 1511, 3902, 1048, 1724, 2760, 467, 4787],
 [4724, 2412, 324, 4556, 305, 3889, 2747],
 [2348, 2020, 3424, 972],
 [108, 2931, 816, 1746, 2736, 3399],
 [1013, 305, 1018, 3506, 2342, 926, 305, 151, 392, 4078],
 [1353,
  4401,
  4165,
  1505,
  3465,
  31,
  1639,
  3705,
  4735,
  2522,
  58,
  231,
  3394,
  1745,
  2747],
 [3695, 2634, 3273, 3684, 4659, 1527, 2461, 2785, 4028, 23, 2352],
 [4467, 3199, 1836, 3063, 3378, 3541, 31, 3322, 4028, 23, 2352],
 [507, 2992, 1841, 3082, 4540, 495, 4037, 2947, 31, 3845],
 [4908, 2957, 4777, 3008, 508, 1896, 3849, 4187],
 [329, 3136, 3371, 2219, 2018, 4782, 2933, 2537, 159, 4640, 3795],
 [1746, 799, 3902, 495, 31, 3378],
 [839, 3053, 1762, 4289, 2060, 4331, 3830, 3622, 4960],
 [2076, 3084, 1427, 2049, 4864, 244, 2293, 4028, 23, 2352],
 [1511, 4100, 1228, 36, 960, 4028, 23, 2352],
 [4389, 1829, 1881, 3335, 4095, 2406, 2159, 2624, 4367, 3108],
 [2383, 473, 2412],
 [524, 4913, 1897, 562, 31, 1845, 960, 2747],
 [4137, 1083, 324, 3213, 2

In [21]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2760  467 4787]
 [   0    0    0 ...  305 3889 2747]
 [   0    0    0 ... 2020 3424  972]
 ...
 [   0    0    0 ... 4028   23 2352]
 [   0    0    0 ... 1245  153  701]
 [   0    0    0 ... 2135 4042 1759]]


In [22]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  657,
       4677,  249, 1511, 3902, 1048, 1724, 2760,  467, 4787])

In [23]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [24]:

len(embedded_docs),y.shape

(18285, (18285,))

In [25]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [26]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [28]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 12s 64ms/step - loss: 0.3452 - accuracy: 0.8332 - val_loss: 0.2118 - val_accuracy: 0.9109
Epoch 2/10
192/192 [==============================] - 15s 79ms/step - loss: 0.1458 - accuracy: 0.9407 - val_loss: 0.1874 - val_accuracy: 0.9229
Epoch 3/10
192/192 [==============================] - 16s 84ms/step - loss: 0.1045 - accuracy: 0.9618 - val_loss: 0.2115 - val_accuracy: 0.9112
Epoch 4/10
192/192 [==============================] - 22s 115ms/step - loss: 0.0774 - accuracy: 0.9720 - val_loss: 0.2170 - val_accuracy: 0.9142
Epoch 5/10
192/192 [==============================] - 18s 94ms/step - loss: 0.0529 - accuracy: 0.9819 - val_loss: 0.2794 - val_accuracy: 0.9176
Epoch 6/10
192/192 [==============================] - 13s 68ms/step - loss: 0.0365 - accuracy: 0.9871 - val_loss: 0.2744 - val_accuracy: 0.9157
Epoch 7/10
192/192 [==============================] - 17s 89ms/step - loss: 0.0274 - accuracy: 0.9908 - val_loss: 0.4021 - val_accuracy

In [29]:
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [30]:
from sklearn.metrics import confusion_matrix

In [31]:
confusion_matrix(y_test,y_pred)

array([[3185,  234],
       [ 316, 2300]], dtype=int64)

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9088649544324772